#Robot Navigation in a Dynamic Environment

##Processing the Input File

In [ ]:
input_file = 'input.txt'

In [ ]:
def read_input(path):
    with open(path, 'r') as f:
        # grid dimensions
        n, m = map(int, f.readline().strip().split())

        # b as the battery life and t as the time steps
        b, t = map(int, f.readline().strip().split())

        # (s_x, s_y) as the starting position of the robot and (d_x, d_y) as the destination position of the robot
        s_x, s_y, d_x, d_y = map(int, f.readline().strip().split())

        grid = [f.readline().strip() for _ in range(n)]

        dynamic_obstacles = {}
        for time_step in range(t):
            obstacles = []
            # Number of obstacles at this time step
            k = int(f.readline().strip())
            for obstacle in range(k):
                obstacles.append(tuple(map(int, f.readline().strip().split())))
            dynamic_obstacles[time_step] = obstacles


    return n, m, b, t, (s_x, s_y), (d_x, d_y), grid, dynamic_obstacles

def print_input(path):
  n, m, b, t, start, destination, grid, dynamic_obstacles = read_input(path)
  print("Grid size:", n, "x", m)
  print("Battery life:", b)
  print("Time steps:", t)
  print("Starting position:", start)
  print("Destination position:", destination)
  print("grid:")
  for row in grid:
      print(row)
  print("Dynamic obstacles:", dynamic_obstacles)


In [ ]:
print_input(input_file)
n, m, b, t, start, destination, grid, dynamic_obstacles = read_input(input_file)

Grid size: 5 x 5
Battery life: 15
Time steps: 3
Starting position: (0, 0)
Destination position: (4, 4)
grid:
.#...
..#..
...#.
.....
.....
Dynamic obstacles: {0: [(1, 2), (3, 3)], 1: [(2, 2)], 2: []}


##Search Algorithms


##General Search Structure: Node class
Node class represent the state of the robot

In [ ]:
class Node:
    def __init__(self, position, parent=None, g_cost=0, h_cost=0, time=0):
        """
        - position (tuple): The (x, y) position of the node on the grid.
        - parent (Node): The parent node, used to reconstruct the path.
        - g_cost (int): The cost from the start node to this node.
        - h_cost (int): The heuristic cost to the goal (used in A*).
        - time (int): The current time.
        """
        self.position = position
        self.parent = parent
        # This is particularly for A* Search Algorithm
        self.g_cost = g_cost
        self.h_cost = h_cost
        self.f_cost = g_cost + h_cost
        self.time = time


    def __eq__(self, other):
        """
        Nodes are considered equal if their positions and time steps match.
        """
        return self.position == other.position and self.time_step == other.time_step

    def __lt__(self, other):
        """
        Override less-than for priority queue comparisons.
        A node is considered less than another if its f value is less.
        """
        return self.f_cost < other.f_cost

    def __repr__(self):
        return f"Node(position={self.position}, f_cost={self.f_cost})"

In [ ]:
# Sample usage
start_node = Node(position=(0, 0), g_cost=0, h_cost=10, time=0)
current_node = Node(position=(1, 0), parent=start_node, g_cost=1, h_cost=8, time=1)

print(current_node)
print(start_node == current_node)
print(start_node < current_node)

Node(position=(1, 0), f_cost=9)
False
False


##General Search Structure: Search class
Here's a general search class which has the essential methods for any search algorithms. Each specific search algorithm required for the robot navigation exercise is implemented as a child of this class.

In [ ]:
import heapq

class Search:
    def __init__(self, grid, start, goal, battery_life, dynamic_obstacles, step_interval):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.battery_life = battery_life
        self.dynamic_obstacles = dynamic_obstacles
        self.step_interval = step_interval
    def heuristic(self, a, b):
        """Calculate Manhattan distance as a heuristic."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def is_valid(self, position, time_step):
        """Check if a move is valid considering static and dynamic obstacles."""
        x, y = position
        n, m = len(self.grid), len(self.grid[0])

        # Check boundaries
        if x < 0 or x >= n or y < 0 or y >= m:
            return False

        # Check static obstacles
        if self.grid[x][y] == '#':
            return False

        # Check dynamic obstacles
        if position in self.dynamic_obstacles.get(time_step, []):
            return False

        return True

    def generate_path(self, node):
        """Genreate the path from the start to the current node."""
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]




##A* Search Algorithm
 The `search` method uses a priority queue (`exploration_queue`) to explore nodes based on their `f_cost` (the sum of the path cost and heuristic estimate to the goal). Dynamic obstacles are handled by associating their movements with discrete time steps, calculated using the total time elapsed (`current_node.time // self.step_interval`). The algorithm validates moves using the `is_valid` method, which checks for static grid boundaries, obstacles, and dynamic obstacles at the relevant time step. The search stops when the goal is reached or when no more paths exist within the constraints, such as battery life, which limits the number of moves. The `explored_nodes` set prevents revisiting nodes already processed for a specific position and time step, ensuring efficiency. If a valid path is found, it is reconstructed using `generate_path` and returned; otherwise, the method returns "No path found."

In [56]:
class AStarSearch(Search):
    def __init__(self, grid, start, goal, battery_life, dynamic_obstacles, step_interval):
        super().__init__(grid, start, goal, battery_life, dynamic_obstacles, step_interval)

    def search(self):
        exploration_queue = []
        explored_nodes = set()
        start_node = Node(position=self.start, g_cost=0, h_cost=self.heuristic(self.start, self.goal), time=0)
        heapq.heappush(exploration_queue, start_node)

        while exploration_queue:
            current_node = heapq.heappop(exploration_queue)

            current_time_step = current_node.time // self.step_interval
            if (current_node.position, current_time_step) in explored_nodes:
                continue

            explored_nodes.add((current_node.position, current_time_step))

            if current_node.position == self.goal:
                return self.generate_path(current_node)

            if current_node.g_cost >= self.battery_life:
                continue

            for move in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                neighbor_position = (current_node.position[0] + move[0], current_node.position[1] + move[1])
                neighbor_time = current_node.time + 1
                neighbor_time_step = neighbor_time // self.step_interval

                if not self.is_valid(neighbor_position, neighbor_time_step):
                    continue

                neighbor = Node(
                    position=neighbor_position,
                    parent=current_node,
                    g_cost=current_node.g_cost + 1,
                    h_cost=self.heuristic(neighbor_position, self.goal),
                    time=neighbor_time
                )

                if (neighbor.position, neighbor_time_step) in explored_nodes:
                    continue

                heapq.heappush(exploration_queue, neighbor)

        return "No path found"


##Testing the search algorithm for the given input:

In [63]:
astar = AStarSearch(grid, start, destination, b, dynamic_obstacles, t)
astar_path = astar.search()
if astar_path == "No path found":
    print(astar_path)
else:
    print("Optimal path:", astar_path)

Optimal path: [(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4)]


##Hill Climbing Search Algorithm
 Hill Climbing algorithm finds a path in a grid by always moving to the first valid neighboring position that improves its heuristic (gets closer to the goal). Starting from the initial position, it looks at all possible moves (up, down, left, right), filters out invalid ones (like obstacles or revisited positions), and checks if they improve the heuristic. If a better position is found, the algorithm moves there and continues. If no better move exists, it stops, assuming it's stuck. The process ends when the goal is reached, the battery runs out, or no valid moves are left, and it returns the path to the goal or a failure reason.

In [54]:
class HillClimbingSearch(Search):
    def __init__(self, grid, start, goal, battery_life, dynamic_obstacles, step_interval):
        super().__init__(grid, start, goal, battery_life, dynamic_obstacles, step_interval)

    def search(self):
        """Implement the Hill Climbing algorithm (first best move)."""
        start_node = Node(position=self.start, g_cost=0, h_cost=self.heuristic(self.start, self.goal), time=0)
        current_node = start_node
        battery_charge = self.battery_life

        explored_nodes = set()

        while current_node.position != self.goal:
            if battery_charge <= 0:
                return "No path found (Battery exhausted)"

            current_time_step = current_node.time // self.step_interval

            explored_nodes.add((current_node.position, current_time_step))

            # Generate neighbors
            neighbors = []
            for move in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                neighbor_position = (current_node.position[0] + move[0], current_node.position[1] + move[1])
                neighbor_time = current_node.time + 1
                neighbor = Node(
                    position=neighbor_position,
                    parent=current_node,
                    g_cost=current_node.g_cost + 1,
                    h_cost=self.heuristic(neighbor_position, self.goal),
                    time=neighbor_time
                )
                neighbors.append(neighbor)

            # Filter valid neighbors
            valid_neighbors = []
            for neighbor in neighbors:
                neighbor_time_step = neighbor.time // self.step_interval
                if self.is_valid(neighbor.position, neighbor_time_step) and \
                        (neighbor.position, neighbor_time_step) not in explored_nodes:
                    valid_neighbors.append(neighbor)

            if not valid_neighbors:
                return "No path found (No valid moves)"

            # Select the first improving neighbor
            improved = False
            for neighbor in valid_neighbors:
                if neighbor.h_cost < current_node.h_cost:
                    current_node = neighbor
                    battery_charge -= 1
                    improved = True
                    break

            if not improved:
                return "No path found (Stuck at local optimum)"

        return self.generate_path(current_node)


##Testing the search algorithm for the given input:

In [64]:
hill_climbing = HillClimbingSearch(grid, start, destination, b, dynamic_obstacles, t)
hill_climbing_path = hill_climbing.search()
if isinstance(hill_climbing_path, str):
    print(hill_climbing_path)
else:
    print("Path found:", hill_climbing_path)

Path found: [(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4)]


##Simulated Annealing Search Algorithm

Simulated Annealing algorithm finds a path in a grid by exploring possible moves, even if they sometimes seem worse, to avoid getting stuck in dead ends. Starting from the initial position, it looks at valid nearby positions and uses a random process to decide whether to move to a new position. If the new position is better, it moves there; if it's worse, it might still move there, but the chance decreases as the "temperature" cools. This way, it balances trying new paths and focusing on the best path. The process stops when the temperature gets too low, the battery runs out, or the goal is reached, and it returns the path or a failure reason.

In [35]:
import math
import random

class SimulatedAnnealingSearch(Search):
    def __init__(self, grid, start, goal, battery_life, dynamic_obstacles, step_interval, initial_temp, cooling_rate, min_temp):
        super().__init__(grid, start, goal, battery_life, dynamic_obstacles, step_interval)
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.min_temp = min_temp

    def search(self):
        """Implement Simulated Annealing for grid navigation."""
        current_node = Node(
            position=self.start,
            g_cost=0,
            h_cost=self.heuristic(self.start, self.goal),
            time=0
        )
        current_energy = self.heuristic(current_node.position, self.goal)
        temperature = self.initial_temp
        battery_charge = self.battery_life

        explored_nodes = set()
        explored_nodes.add((current_node.position, current_node.time // self.step_interval))

        while temperature > self.min_temp and battery_charge > 0:
            # Generate a random neighbor
            neighbors = []
            for move in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                neighbor_position = (current_node.position[0] + move[0], current_node.position[1] + move[1])
                neighbor_time = current_node.time + 1
                neighbor_time_step = neighbor_time // self.step_interval

                if self.is_valid(neighbor_position, neighbor_time_step) and \
                        (neighbor_position, neighbor_time_step) not in explored_nodes:
                    neighbor = Node(
                        position=neighbor_position,
                        parent=current_node,
                        g_cost=current_node.g_cost + 1,
                        h_cost=self.heuristic(neighbor_position, self.goal),
                        time=neighbor_time
                    )
                    neighbors.append(neighbor)

            if not neighbors:
                return "No path found (No valid moves)"

            # Select a random neighbor
            random_neighbor = random.choice(neighbors)
            random_neighbor_energy = random_neighbor.h_cost
            delta_energy = random_neighbor_energy - current_energy

            # Decide whether to accept the neighbor
            if delta_energy < 0 or random.uniform(0, 1) < math.exp(-delta_energy / temperature):
                current_node = random_neighbor
                current_energy = random_neighbor_energy
                explored_nodes.add((current_node.position, current_node.time // self.step_interval))
                battery_charge -= 1

            # Cool down
            temperature *= self.cooling_rate

            if current_node.position == self.goal:
                return self.generate_path(current_node)

        return "No path found"


##Testing the search algorithm for the given input:

In [65]:
initial_temp = 50
cooling_rate = 0.9
min_temp = 1e-3

simulated_annealing = SimulatedAnnealingSearch(
    grid=grid,
    start=start,
    goal=destination,
    battery_life=b,
    dynamic_obstacles=dynamic_obstacles,
    step_interval=t,
    initial_temp=initial_temp,
    cooling_rate=cooling_rate,
    min_temp=min_temp
)

simulated_annealing_path = simulated_annealing.search()

if isinstance(simulated_annealing_path, str):
    print(simulated_annealing_path)
else:
    print("Path found:", simulated_annealing_path)


No path found


##Tabu Search Algorithm

Starting from an initial position, the algorithm generates all valid neighbors by simulating moves in four directions and filtering out invalid positions based on constraints. A tabu list is maintained to prevent revisiting recently explored positions, ensuring that the search avoids cycling and explores new areas. The algorithm selects the best non-tabu neighbor based on a heuristic function (Manhattan distance) and updates the current state accordingly. If a neighbor on the tabu list offers a better solution than the best-known state, the tabu restriction is overridden (aspiration criteria). The search continues until the goal is reached, the battery is exhausted, or the maximum number of iterations is reached. If successful, the algorithm returns the optimal path; otherwise, it provides a failure reason.

In [59]:
class TabuSearch(Search):
    def __init__(self, grid, start, goal, battery_life, dynamic_obstacles, step_interval, tabu_list_size, max_iterations):
        super().__init__(grid, start, goal, battery_life, dynamic_obstacles, step_interval)
        self.tabu_list_size = tabu_list_size
        self.max_iterations = max_iterations

    def search(self):
        """Implement the Tabu Search algorithm."""
        current_node = Node(
            position=self.start,
            g_cost=0,
            h_cost=self.heuristic(self.start, self.goal),
            time=0
        )
        best_node = current_node
        battery_remaining = self.battery_life
        tabu_list = []
        explored_nodes = set()

        for iteration in range(self.max_iterations):
            if current_node.position == self.goal:
                return self.generate_path(current_node)

            if battery_remaining <= 0:
                return "No path found (Battery exhausted)"

            current_time_step = current_node.time // self.step_interval

            explored_nodes.add((current_node.position, current_time_step))

            # Generate neighbors
            neighbors = []
            for move in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                neighbor_position = (current_node.position[0] + move[0], current_node.position[1] + move[1])
                neighbor_time = current_node.time + 1
                neighbor_time_step = neighbor_time // self.step_interval

                if self.is_valid(neighbor_position, neighbor_time_step) and \
                        (neighbor_position, neighbor_time_step) not in explored_nodes:
                    neighbor = Node(
                        position=neighbor_position,
                        parent=current_node,
                        g_cost=current_node.g_cost + 1,
                        h_cost=self.heuristic(neighbor_position, self.goal),
                        time=neighbor_time
                    )
                    neighbors.append(neighbor)

            if not neighbors:
                return "No path found (No valid moves)"

            best_neighbor = None
            best_neighbor_score = float('inf')
            for neighbor in neighbors:
                if neighbor.position not in tabu_list or neighbor.h_cost < best_node.h_cost:
                    if neighbor.h_cost < best_neighbor_score:
                        best_neighbor = neighbor
                        best_neighbor_score = neighbor.h_cost

            if best_neighbor is None:
                return "No path found (All neighbors are tabu)"

            # Update the current node
            current_node = best_neighbor
            battery_remaining -= 1

            # Update the best solution found
            if current_node.h_cost < best_node.h_cost:
                best_node = current_node

            tabu_list.append(current_node.position)
            if len(tabu_list) > self.tabu_list_size:
                tabu_list.pop(0)

        if best_node.position == self.goal:
            return self.generate_path(best_node)
        else:
            return "No path found (Max iterations reached)"


##Testing the search algorithm for the given input:

In [66]:
tabu_list_size = 2
max_iterations = 100

tabu_search = TabuSearch(
    grid=grid,
    start=start,
    goal=destination,
    battery_life=b,
    dynamic_obstacles=dynamic_obstacles,
    step_interval=t,
    tabu_list_size=tabu_list_size,
    max_iterations=max_iterations
)

tabu_path = tabu_search.search()

if isinstance(tabu_path, str):
    print(path)
else:
    print("Path found:", tabu_path)


Path found: [(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4)]


##Final Output:

In [68]:
def print_result(path):
  if isinstance(path, str):
    print("No path found")
  else:
    print(len(path))
    for move in path:
      print(move[0], ' ', move[1])
print("A* Algorithm:")
print_result(astar_path)
print("Hill Climbing:")
print_result(hill_climbing_path)
print("Simulated Annealing:")
print_result(simulated_annealing_path)
print("Tabu Search:")
print_result(tabu_path)

A* Algorithm:
9
0   0
1   0
1   1
2   1
3   1
3   2
3   3
3   4
4   4
Hill Climbing:
9
0   0
1   0
1   1
2   1
3   1
3   2
3   3
3   4
4   4
Simulated Annealing:
No path found
Tabu Search:
9
0   0
1   0
1   1
2   1
3   1
3   2
3   3
3   4
4   4
